## SOM_CV_data_swapSOM

This notebook trains the SOMs from the input data where there is a given GTD (either era5 or UK-ESM-1-0-LL piControl). From a specified number of folds in the cross-validation, the function trains and plots the SOM and associated trends from the training data. The SOM is then applied to the test dataset. This is done for each specified set of years.

Here the codebook vectors trained from the ERA5 SOM is applied to the UKESM1-0-LL data to see if it can produce a better picture for the distribution of weather patterns.

The opposite will also be tried as a way of testing the UKESM SOM.



In [1]:
import numpy as np
import scipy
import netCDF4
import matplotlib as mpl
mpl.use("Agg", warn=False)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
import pandas as pd
import os
os.environ["PROJ_LIB"] = "/rds/general/user/kc1116/home/anaconda3/envs/zeus/share/proj"
import matplotlib as mpl
import matplotlib
import matplotlib.colors as colors
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import rcParams
from mpl_toolkits.mplot3d import Axes3D
import somoclu
import cartopy
import xarray
import xarray as xr
import glob
import math
from mpl_toolkits.basemap import Basemap as bm
from itertools import groupby
from scipy import stats
import cartopy.crs as ccrs
from statsmodels.stats.multitest import (multipletests, fdrcorrection,
                                         fdrcorrection_twostage,
                                         NullDistribution,
                                         local_fdr)
from scipy.stats import spearmanr, kendalltau, pearsonr
from functools import wraps
import errno
import os
import signal
import SOM_trends_funcs as SOM_fn

In [2]:
#Function to plot the SOM trends
#added values to modify Gerald's function (issues with date formatting)
def saveSOMTrends(som_yr, dates, n_rows, n_columns, savefig_trends, colormap_str, mult_test_method, mdl, train_input=False, bmus=None, JJA_days=104, yrst=1979, yrend=2019, samp=10, plot=True, showplot=False):
    """
    Plots the trends (occurrence, persistence, max duration) of the SOM patterns generated from the GPH data. 
    
    Parameters
    ----------
    som_yr    : somoclu.train.Somoclu
        The trained SOM (somoclu object)
    seas_yr   : xarray.DataArray
        Data array containing GPH data and information on time, latitude, longitude variables in the selected time range, season and region
        
    Returns
    ---------
    grad_list : list 
        List of the gradients of the pattern occurrence trends computed by linear least-squares regression 
    all_occur : list 
        List of occurrences per day in the trained dataset for each SOM pattern
    """
    if train_input==False:
        #calculate the best matching unit
        #Extract pattern data (occurence, persistence, max duration)
        #Store each pattern data as a binary list of len(dates), i.e. [1,0,0,0,1...,1]
        bmus = som_yr.bmus
    
    global all_occur
    all_occur = [[] for i in range(max(n_rows,n_columns))]
    for a in range(n_rows):
        for b in range(n_columns): 
            curr_occur = []
            for c in bmus:
                if train_input==False:
                    if c.tolist() == [b,a]: #### need to swap the indices here!! #b'a
                        curr_occur.append(1) #This pattern occured on this date
                    else:
                        curr_occur.append(0) #This pattern didn't occur 
                else:
                    if c == [b,a]: #### need to swap the indices here!! #b'a
                        curr_occur.append(1) #This pattern occured on this date
                    else:
                        curr_occur.append(0) #This pattern didn't occur                     
            all_occur[a].append(curr_occur)
    #Loop through each set of circulation occurence data
    da_arr, rownum_arr, colnum_arr = [], [], []
    
   
    
    ##### Plotting - can put in a different function
    #####
    if plot:
        #Plotting axes the extracted circulation data
        if n_columns == 1:
            f, axes1 = plt.subplots(nrows=n_rows, ncols=n_columns+1, figsize=(10,10))
        elif n_rows == 1:
            f, axes1 = plt.subplots(nrows=n_rows+1, ncols=n_columns, figsize=(10,10))
        else:
            f, axes1 = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(10,10))
        f.subplots_adjust(hspace=0.2, wspace=0.2)
    test_num = 8 #number of p value tests in family of hypotheses (occ, persis, max_dur and event_no * 2)
    #pvalue and gradient matrices for each node
    pval_mat, grad_mat = np.zeros((n_rows,n_columns,test_num)), np.zeros((n_rows,n_columns,test_num))    
    grad_list = []
    for row in range(n_rows):  
        for col in range(n_columns):
            #Need some exception handling for rare cases where any of the patterns doesnt occur in a year
            #NOTE the swapped indices [col][row] here! Doulbe check with Joy's code, see if node number can be confirmed
            #print(f"all_occur[row][col] = {all_occur[row][col]}")
            #all_occur[row][col] is the relevant matrix for investigating case studies
            cum_data = pd.Series(np.array(all_occur[row][col]), index=dates.values)
            #print(f"cum_data = {cum_data}")
            #to generate the years in the dataset, since cum_data.index.year has issues
            #print(f"JJA_days = {JJA_days}")       
            #print(f"cum_data.shape = {cum_data.shape}")
            years_num=int(cum_data.shape[0]/JJA_days)
            #print(f"cum_data.shape[0]/JJA_days = {cum_data.shape[0]}/{JJA_days} = {cum_data.shape[0]/JJA_days}")
            #print(f"years_num = {years_num}")
            years_zeros = np.zeros((years_num*JJA_days))
            if type(yrst) == int or type(yrst) == float:
                arr_gen = np.array([years_zeros[JJA_days*i:JJA_days*(i+1)]+i+yrst for i in range(years_num)
                               ]).reshape(years_num*JJA_days)
            else: #yrst an array
                print("yrst an array")
                arr_gen = np.array([years_zeros[JJA_days*i:JJA_days*(i+1)]+i+int(yrst.values) for i in range(years_num)
                               ]).reshape(years_num*JJA_days)
            #print(f"len(arr_gen) = {len(arr_gen)}")
            sort_to_year = cum_data.groupby(arr_gen).apply(list)
            year = np.array(sort_to_year.keys().tolist()) #[1979, 1980,...]
            occ = [sum(d) for d in sort_to_year]  #[25,40,...]
            # Count streaks 
            streaks = [[sum(1 for i in g) for k,g in groupby(x) if k == 1] for x in sort_to_year]
            streak_count = [[(k, sum(1 for i in g)) for k,g in groupby(sorted(x)) ] for x in streaks]
            
            #Persistence (if statement to avoid division by zero )
            persis = [ sum([x[0]*x[1] for x in year])/sum([x[1] for x in year]) if sum([x[1] for x in year]) != 0 else 0 for year in streak_count ]
            #Max Duration 
            max_dur = [ year[-1][0] if year != [] else 0 for year in streak_count]
            ##want to store occ, persis, max_dur and ev_no in a xarray Dataset/DataArray to save, with the time file showing the years
            ##need to store this for each node (36 different 1d arrays for each model, plus the timestamp)

            def len_iter(items):
                return sum(1 for _ in items)            
            
            def consecutive_one_len(data):
                return len(list(len_iter(run) for val, run in groupby(data) if val))            
            event_no = [consecutive_one_len(d) for d in sort_to_year] 
            
            store = [occ, persis, max_dur, event_no]
            
            da = xarray.DataArray(store, dims = ('SOM_trend_metrics', "year"))
            da['year'] = year
            da['SOM_trend_metrics'] = ['occ', 'persis', 'max_dur', 'event_no']
            da_arr.append(da)
            #store the DataArray and the accompanying row and column number of the SOM pattern            
            rownum_arr.append(row)
            colnum_arr.append(col)
            
            if plot:
                def len_iter(items):
                    return sum(1 for _ in items)            

                def consecutive_one_len(data):
                    return len(list(len_iter(run) for val, run in groupby(data) if val))            
                event_no = [consecutive_one_len(d) for d in sort_to_year]      
                #event_no = [sum(d) for d in sort_to_year]        
                #np.ones((len(sort_to_year)))*sum(sort_to_year)
                #Fitting via Linear Least Squares Regression (With Serial Correlation)
                def fit(m,x,c):  #Simple linear function
                    return m*x + c

                #Occurrence 
                m0, c0, r0, p0, err0 = scipy.stats.linregress(year,occ)
                m1, c1, r1, p1, err1 = scipy.stats.linregress(year[samp:],occ[samp:])
                fit_occ = [fit(m0,x,c0) for x in year]
                fit_occ1 = [fit(m1,x,c1) for x in year[samp:]]
                grad_list.append(m0)

                #Persistence
                m2, c2, r2, p2, err2 = scipy.stats.linregress(year,persis)
                m3, c3, r3, p3, err3 = scipy.stats.linregress(year[samp:],persis[samp:])
                fit_per = [fit(m2,x,c2) for x in year]
                fit_per1 = [fit(m3,x,c3) for x in year[samp:]]

                #Max duration
                m4, c4, r4, p4, err4 = scipy.stats.linregress(year,max_dur)
                m5, c5, r5, p5, err5 = scipy.stats.linregress(year[samp:],max_dur[samp:])
                fit_dur = [fit(m4,x,c4) for x in year]
                fit_dur1 = [fit(m5,x,c5) for x in year[samp:]]

                #Event number
                m6, c6, r6, p6, err6 = scipy.stats.linregress(year,event_no)
                m7, c7, r7, p7, err7 = scipy.stats.linregress(year[samp:],event_no[samp:])
                fit_evno = [fit(m6,x,c6) for x in year]
                fit_evno1 = [fit(m7,x,c7) for x in year[samp:]]
                
                #Plot results
                row, col = int(row), int(col)
                occ, persis, max_dur, event_no = np.array(occ), np.array(persis), np.array(max_dur), np.array(event_no)
                
                axes1[row][col].scatter(year, occ, c='k', s=3, label="Occurrences") #plot occurrences vs years
                axes1[row][col].plot(year, fit_occ, c='k')
                axes1[row][col].plot(year[samp:], fit_occ1, '--', c='k')

                axes1[row][col].scatter(year, persis, c='b', s=3, label="Persistence") #plot persistence vs years
                axes1[row][col].plot(year, fit_per, c='b')
                axes1[row][col].plot(year[samp:], fit_per1, '--', c='b')

                axes1[row][col].scatter(year, max_dur, c='r', s=3, label="Max Duration") #plot max duration vs years
                axes1[row][col].plot(year, fit_dur, c='r')
                axes1[row][col].plot(year[samp:], fit_dur1, '--', c='r')

                axes1[row][col].scatter(year, event_no, c=(0.5,0.5,0.5), s=3, label="Event no") #plot event number vs years
                axes1[row][col].plot(year, fit_evno, c=(0.5,0.5,0.5))
                axes1[row][col].plot(year[samp:], fit_evno1, '--', c=(0.5,0.5,0.5))         

                if n_rows*n_columns > 5:
                    font_size_text = 8
                    font_size_title = 9 
                else:
                    font_size_text = 11
                    font_size_title = 12                  
                font_size_suptitle = 14
                #corrected p values
                (pval_occ, pval_occ_samp, pval_persis, pval_persis_samp, 
                 pval_max_dur, pval_max_dur_samp, pval_event_no, pval_event_no_samp) = (SOM_fn.lmtrendtest(year, occ), SOM_fn.lmtrendtest(year[samp:], occ[samp:]),
                                                                                        SOM_fn.lmtrendtest(year, persis), SOM_fn.lmtrendtest(year[samp:], persis[samp:]),
                                                                                        SOM_fn.lmtrendtest(year, max_dur), SOM_fn.lmtrendtest(year[samp:], max_dur[samp:]),
                                                                                        SOM_fn.lmtrendtest(year, event_no), SOM_fn.lmtrendtest(year[samp:], event_no[samp:]))

                #store the calculated p values in a matrix
                pval_mat[row,col,:] = [pval_occ, pval_occ_samp, pval_persis, pval_persis_samp, pval_max_dur, pval_max_dur_samp, pval_event_no, pval_event_no_samp]
                grad_mat[row,col,:] = [m0,m1,m2,m3,m4,m5,m6,m7]
    
    if plot:
        if mult_test_method == "none":
            print("No multiple hypothesis correction applied")
        elif mult_test_method == "k-FWER":    
            #test for the k-familywise error rate using the Bonferroni correction
            pval_arr = SOM_fn.Bonferroni_kFWER(pval_mat[:,:,:].flatten(), k = 20)
            pval_mat = pval_arr.reshape((n_rows,n_columns,test_num)) 
        else:
            pval_arr = multipletests(pval_mat[:,:,:].flatten(), alpha=0.05, method=mult_test_method, is_sorted=False, returnsorted=False)[1]  
            pval_mat = pval_arr.reshape((n_rows,n_columns,test_num)) 

        for row in range(n_rows):  
            for col in range(n_columns):            

                m0,m1,m2,m3,m4,m5,m6,m7 = grad_mat[row,col,:]
                p0,p1,p2,p3,p4,p5,p6,p7 = pval_mat[row,col,:]            

                #Legend
                #calculate the corrected p values and create the strings to reduce the bold text
                m0_str, m1_str, m2_str, m3_str, m4_str, m5_str, m6_str, m7_str = (SOM_fn.pval_str(m0, p0), SOM_fn.pval_str(m1, p1), 
                                                                  SOM_fn.pval_str(m2, p2), SOM_fn.pval_str(m3, p3), 
                                                                  SOM_fn.pval_str(m4, p4), SOM_fn.pval_str(m5, p5),
                                                                  SOM_fn.pval_str(m6, p6), SOM_fn.pval_str(m7, p7))
                axes1[row][col].text(0.16, 0.99, "Occurrence", transform=axes1[row][col].transAxes, fontsize=font_size_title, verticalalignment='top', horizontalalignment='center')
                axes1[row][col].text(0.16, 0.93, m0_str,
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center')
                axes1[row][col].text(0.16, 0.87, m1_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center')

                axes1[row][col].text(0.49, 0.99, "Persistence", transform=axes1[row][col].transAxes, fontsize=font_size_title, verticalalignment='top',horizontalalignment='center', color='b')
                axes1[row][col].text(0.49, 0.93, m2_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color='b')
                axes1[row][col].text(0.49, 0.87, m3_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color='b')

                axes1[row][col].text(0.83, 0.99, "Max duration", transform=axes1[row][col].transAxes, fontsize=font_size_title, verticalalignment='top', horizontalalignment='center', color='r')
                axes1[row][col].text(0.83, 0.93, m4_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color='r')
                axes1[row][col].text(0.83, 0.87, m5_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color='r')      

                axes1[row][col].text(0.16, 0.82, f"Event no", transform=axes1[row][col].transAxes, fontsize=font_size_title, verticalalignment='top', horizontalalignment='center', color=(0.5,0.5,0.5))
                axes1[row][col].text(0.16, 0.76, m6_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color=(0.5,0.5,0.5))
                axes1[row][col].text(0.16, 0.70, m7_str, 
                                     transform=axes1[row][col].transAxes, fontsize=font_size_text, verticalalignment='top', horizontalalignment='center', color=(0.5,0.5,0.5))               

                axes1[row][col].set_ylim((0,55))            
            f.suptitle(f'Time series of circulation trends \n {mdl} {yrst}-{yrend}', fontsize=14)
        f.savefig(savefig_trends, bbox_inches="tight", dpi=300) 
        if showplot == False:
            plt.close();
    #####        
    #####        
            
    #concatenate the array
    da_xr = xarray.concat(da_arr, dim = ("node"))
    #add a new coordinates to specify the row and column number for each layer
    da_xr = da_xr.assign_coords(
        rownum=('node', rownum_arr))
    da_xr = da_xr.assign_coords(
        colnum=('node', colnum_arr))           
            
    return da_xr, all_occur



In [3]:

def save_SOM_data(zg_file, yrst, yrend, season, region, init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling,
            savefig_title, savefig_trends, n_rows, n_columns, samp, zg_str, colormap_str, suptitle, mult_test_method, #yrst=1979, yrend=2013
                data_yr_reshaped, lat_str, lon_str, units, mdl, train_input = False, som_yr = None, JJA_days=104, save_SOM_str = False, save_SOM_trends_str = False, showplot=False):
    """
    Save the data for all the climate models
    """
    #Get the prepared data
    #issue with prepData function so load information beforehand
    lats, lons, dates = zg_file[lat_str], zg_file[lon_str], data_yr_reshaped['time']
    yrst, yrend = int(data_yr_reshaped['time.year'].min()), int(data_yr_reshaped['time.year'].max())
    nr_lat, nr_lon = len(lats), len(lons)
    m = bm(projection='cyl',llcrnrlat=lats[0],urcrnrlat=lats[-1],llcrnrlon=lons[0],urcrnrlon=lons[-1],resolution='l')
    #print(f"save_SOM data_yr_reshaped.shape = {data_yr_reshaped.shape}")
    #Train the SOM if not inputting an already trained SOM pattern
    if train_input == False:
        print("training")
        som_yr = somoclu.Somoclu(n_columns, n_rows, maptype="planar",compactsupport=True,initialization=f"{init}", neighborhood=f"{neigh}", std_coeff=std)
        som_yr.train(data_yr_reshaped,epochs=ep,radius0=rad_0,radiusN=rad_N,radiuscooling=f"{rad_cooling}",
                 scale0=scale_0,scaleN=scale_N,scalecooling=f"{scale_cooling}")
        bmus=som_yr.bmus
    else:
        #already have som_yr but need to calculate bmus
        codebook_da = xarray.DataArray(som_yr.codebook, name = "codebook", dims = ("row", "col", "latlon"))
        codebook_da_reshaped = codebook_da.values.reshape(n_rows*n_columns, data_yr_reshaped.shape[1])
        codebook_da_reshaped_xr = xr.DataArray(codebook_da_reshaped, name = f"codebook_reshaped")
        codebook_da_reshaped_xr = codebook_da_reshaped_xr.rename(dim_0="rowcol").rename(dim_1="latlon_flat")
        bmus = SOM_fn.Identify_BMU_from_codebook(codebook_da_reshaped_xr, data_yr_reshaped, n_rows, n_columns)
        
    #Plot the trained SOM nodes 
    g, axes = plt.subplots(nrows=n_rows, ncols=n_columns, figsize=(10,5))
    g.subplots_adjust(hspace=0.2, wspace=0.2)
    #Store the frequency data in a list
    freq_list = []
    for i in range(n_rows): 
        for j in range(n_columns):
            try:
                ax = axes[i][j]
                node = som_yr.codebook[i][j]
            except: #doesn't work when not indexed, when one of n_rows or n_columns = 1
                if n_rows > n_columns:
                    ax = axes[i]
                    node = som_yr.codebook[j]
                else:
                    ax = axes[i]
                    node = som_yr.codebook[i]
            #print(f"node = {node}")
            #print(f"node.shape = {node.shape}")
            node_orig = node.reshape(nr_lat,nr_lon)            
            
            SOM_fn.plot_field(m, node_orig, lats, lons, -330, 330, 30, ncols = n_columns, nrows = n_rows,  #for pv use -330/250, 330/250, 30/250 as levels
                       ax=ax, grid=False, cmap=plt.get_cmap(colormap_str), fig=g, row=i, col=j)
            #Pattern frequencies
            if train_input == False:
                freq = sum([1 for pat in som_yr.bmus if pat.tolist() == [j,i]])/float(len(som_yr.bmus))*100 #note the flipped indices
            else:
                #print(f"pat = {[j,i]}")
                freq = sum([1 for pat in bmus if pat == [j,i]])/float(len(bmus))*100 #note the flipped indices
                #print(f"freq = {freq}")
            freq_list.append(freq)
            props = dict(boxstyle='round', facecolor='white', alpha=0.8)
            ax.text(0.05, 0.95, str(round(freq,1))+"%", transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
            #ax1.ticklabel_format(style="plain")      
    suptitle=f"zg LTDM anom {mdl} {yrst}-{yrend} JJA ({units})"            
    g.suptitle(f"{suptitle}", fontsize=10)    
    ax.ticklabel_format(style="plain")      
    if save_SOM_str != False:
        print(f"save_SOM_str = {save_SOM_str}")
        g.savefig(save_SOM_str, bbox_inches="tight", dpi = 300)
    if showplot == False:
        plt.close();
        
    da_xr_trend, all_occur = saveSOMTrends(som_yr, dates, n_rows, n_columns, savefig_trends, colormap_str,
                                mult_test_method, mdl, train_input=train_input, bmus=bmus, JJA_days=JJA_days, yrst=yrst, yrend=yrend, samp=samp)#, save_SOM_trends_str=save_SOM_trends_str)
    return da_xr_trend, all_occur, som_yr


Run cell below to train z500 on ERA5 data and use the resulting map for UKESM1-0-LL for a given node number

In [4]:
    
def train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns):
    """
    Train the SOM maps for what 
    """
    file_zg_tot_train = xarray.open_dataset(file_zg_str_train).squeeze()

    if "psl" in file_zg_str_train:
        zg_str = "psl"
        units = "Pa"
        data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/psl/"
    "psl_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd_reshaped.nc")     
    if "msl" in file_zg_str_train:
        zg_str = "msl"
        units = "Pa"
        data_yr_reshaped_str = f"/rds/general/project/nowack_graven/live/carl/era5/mean_sea_level_pressure/mslp_ERA5_{yrst}-{yrend}_EUR_JJAextd_LTDMdaymean_anom_reshaped2.nc"
    if "zg" in file_zg_str_train:
        zg_str = "z"
        units = "hPa"
        data_yr_reshaped_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_sort_reshaped.nc"
        if "r180x91" in file_zg_str_train:
            data_yr_reshaped_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_r180x91_reshaped.nc"
    if "UKESM" in file_zg_str_train:
        if "zg" in file_zg_str_train:
            zg_str = "zg"
            data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/"
                "500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR_JJAextd_dtrnd_reshaped.nc")
            data_yr_reshaped_train = xr.open_dataset(data_yr_reshaped_str)['data_yr_reshaped']
        if "psl" in file_zg_str_train:
            zg_str = "psl"
            data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/psl/"
                "psl_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR_JJAextd_dtrnd_reshaped.nc")
            data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)['data_yr_reshaped']      
    if "era5" in data_yr_reshaped_str:
        if "r180x91" not in data_yr_reshaped_str:
            data_yr_reshaped_train = xr.open_dataset(data_yr_reshaped_str)[f'era5_{zg_str}_reshaped']   
        else:
            data_yr_reshaped_train = xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_r180x91_reshaped.nc")['__xarray_dataarray_variable__']

    if "r180x91" not in file_zg_str_train:
        file_zg_tot_train = file_zg_tot[zg_str]

    if "latitude" in str(file_zg_tot_train.coords):    
        lat_str, lon_str = "latitude", "longitude"
    else:
        lat_str, lon_str = "lat", "lon"

    if "era5" in file_zg_str_train:
        mdl, ens = "era5", "reanal"
        mdl_ens_str = f"{mdl}_{ens}"
        mdl_str = "era5"
        swap_str = "_tot"#"_swapERA5"    
        mip = "reanal"
    if "UK" in file_zg_str_train:
        mdl, ens = "UKESM1-0-LL", "piControl"
        mdl_ens_str = f"{mdl}_{ens}"
        mdl_str = f"{mdl}_{ens}"
        mip = "cmip6"
        swap_str = "_tot"#"_swapUKESM1-0-LL"    
        lat_str, lon_str = "lat", "lon"
    lats, lons = np.arange(30,77), np.arange(-10,41)
    #need to adjust the longitude coordinate if it runs from 0 - 360 E instead of -180 to +180 E
    file_zg_tot_train = SOM_fn.da_lon_adj(file_zg_tot_train)
    if "UKESM" in file_zg_str_train:
        file_zg_tot_train = file_zg_tot_train.sel(time = np.isin(file_zg_tot_train['time.dayofyear'], np.arange(147,245)))
    elif "era5" in file_zg_str_train and "r180x91" not in file_zg_str_train:
        JJA_extd_daymonth_xr = xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/day_month_1979-2019_JJAextd.nc")['JJA_extd']
        #zg_file['day_month'] = xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/day_month_1979-2019_nolp.nc")['day_month']
        day_month = xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/day_month_1979-2019_nolp.nc")['day_month']
        file_zg_tot_train = file_zg_tot_train.sel(latitude = np.isin(file_zg_tot_train[lat_str], lats), 
                                  longitude = np.isin(file_zg_tot_train[lon_str], lons))
        #if "msl" in file_zg_str_train:
        #    file_zg_tot = file_zg_tot.sel(time = np.isin(day_month, JJA_extd_daymonth_xr))
        #.sel(time = np.isin(file_zg_tot['time.year'], np.arange(yrst_train, yrst_train+yrnum_train+1), invert=True))[zg_str]
    num_days, num_lats, num_lons = file_zg_tot_train['z'].shape

    yrst, yrend = int(file_zg_tot_train['time.year'].min()), int(file_zg_tot_train['time.year'].max())
    samp = int((yrst-yrend)/2)
    init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling = "pca", "gaussian", 0.5, 50, 1, 0, "linear", 0.1, 0.01, "exponential"
    region, season, domain = "Europe", "JJA_extd", "EUR"


    dir_str = "/rds/general/project/nowack_graven/live/carl_som_index/data"
    savefig_SOM = (f"{dir_str}/{mdl_str}/{zg_str}/plots/"
                f"SOM_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.png")
    savefig_trends = (f"{dir_str}/{mdl_str}/{zg_str}/plots/"
                f"SOMtrends_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.png")
    SOM_trend_str_train = (f"{dir_str}/{mdl_str}/{zg_str}/"
                f"SOM_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.nc")
    SOM_data_occ_str_train = (f"{dir_str}/{mdl_str}/{zg_str}/"
                f"SOM_data_occur_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.nc")
    SOM_trend_str_cv = (f"{dir_str}/{mdl_str}/{zg_str}/"
                f"SOM_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.nc")
    SOM_data_occ_str_cv = (f"{dir_str}/{mdl_str}/{zg_str}/"
                f"SOM_data_occur_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.nc")

    #print(f"SOM_data_occ_str_train = {SOM_data_occ_str_train}")
    #print(f"SOM_data_occ_str_cv = {SOM_data_occ_str_cv}")    
    
    JJA_days = int(len(np.unique(file_zg_tot_train['time.dayofyear'])))
    if "era5" in file_zg_str_train and "msl" not in file_zg_str_train:
        JJA_days = JJA_days-1
    colormap_str = "seismic"
    suptitle=f"zg LTDM anom {mdl} {yrst}-{yrend} JJA extd ({units})"
    mult_test_method = "fdr_bh"#"k-FWER"#"none"#"fdr_bh"
    
    #print(f"training original SOM data on {file_zg_str_train}")
    da_xr_trend_train, all_occur_train, som_yr_train = save_SOM_data(file_zg_tot_train, yrst, yrend, 
                        season, region, init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling,
                        savefig_SOM, savefig_trends, n_rows, n_columns, samp, zg_str, colormap_str, suptitle, mult_test_method, 
                        data_yr_reshaped_train, lat_str, lon_str, units, mdl_ens_str, train_input = False, JJA_days=JJA_days, save_SOM_str=savefig_SOM, showplot=True)
    da_xr_trend_train.to_netcdf(SOM_trend_str_train)
    print(f"saved trends in {SOM_trend_str_train}")
#    print(f"all_occur_train = {all_occur_train}")
    da = xarray.DataArray(all_occur_train, name = "SOM_data", dims = ("row", "col", "time"))
    da['time'] = file_zg_tot_train['time']
    da.to_netcdf(SOM_data_occ_str_train)
    print(f"saved data_occ in {SOM_data_occ_str_train}")
#    da_xr_trend_train
#    /rds/general/project/nowack_graven/live/carl_som_index/data/zg/
    ##add the code below - just running again here since I neglected to save the training data files 29/10/20
    print(f"test the trained SOM on {file_zg_str_test}")
    yrnum_train=10 #irrelevant value here since variable is simply apssed through the function
    SOM_calc_samp_swap(file_zg_str_test, n_rows, n_columns, yrnum_train, som_yr_train_swap = som_yr_train, showplot = True)
    return


In [5]:

def SOM_calc_samp_swap(file_zg_str, n_rows, n_columns, yrnum_train, som_yr_train_swap = None, showplot = False):
    """
    Calculate and save the trained arrangement of SOM nodes for a set of subsamples of the data
    Compare the perfrmance of the trained SOM for ERA5 with UK-ESM-1-0-LL when applying each to the other
    One expects the performance - this tells us how important the files are
    """
    file_zg_tot = xarray.open_dataset(file_zg_str).squeeze()
    
    yrst, yrend = int(file_zg_tot['time.year'].min()), int(file_zg_tot['time.year'].max())
    yrst_train_arr = [yrst+i*yrnum_train for i in range(int((yrend - yrst)/yrnum_train))]    
    
    k = int((yrend - yrst)/yrnum_train)
    #print(f"{k}-fold cv")    
    if "psl" in file_zg_str:
        zg_str = "psl"
        units = "Pa"
        data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/psl/"
 "psl_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd_reshaped.nc")     
    if "msl" in file_zg_str:
        zg_str = "msl"
        units = "Pa"
        data_yr_reshaped_str = f"/rds/general/project/nowack_graven/live/carl/era5/mean_sea_level_pressure/mslp_ERA5_{yrst}-{yrend}_EUR_JJAextd_LTDMdaymean_anom_reshaped2.nc"
    if "zg" in file_zg_str:
        zg_str = "z"
        units = "hPa"
        data_yr_reshaped_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_sort_reshaped.nc"
        if "r180x91" in file_zg_str:
            data_yr_reshaped_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_r180x91_reshaped.nc"
    if "UKESM" in file_zg_str:
        if "zg" in file_zg_str:
            zg_str = "zg"
            data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/"
                "500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR_JJAextd_dtrnd_reshaped.nc")
            data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)['data_yr_reshaped']
        if "psl" in file_zg_str:
            zg_str = "psl"
            data_yr_reshaped_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/psl/"
                "psl_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR_JJAextd_dtrnd_reshaped.nc")
            data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)['data_yr_reshaped']      
    if "era5" in data_yr_reshaped_str and "r180x91" not in data_yr_reshaped_str:
        data_yr_reshaped = xr.open_dataset(data_yr_reshaped_str)[f'era5_{zg_str}_reshaped']   
    if "r180x91" and "era5" not in file_zg_str:
        file_zg_tot = file_zg_tot[zg_str]

    lat_str, lon_str = "latitude", "longitude"
    if "era5" in file_zg_str:
        mdl, ens = "era5", "reanal"
        mdl_ens_str = f"{mdl}_{ens}"
        mip = "reanal"
    if "UK" in file_zg_str:
        mdl, ens = "UKESM1-0-LL", "r1i1p1f2"
        mdl_ens_str = f"{mdl}_{ens}"
        mip = "cmip6"
        lat_str, lon_str = "lat", "lon"
    lats, lons = np.arange(30,76), np.arange(-10,41)
    #need to adjust the longitude coordinate if it runs from 0 - 360 E instead of -180 to +180 E
    file_zg_tot=SOM_fn.da_lon_adj(file_zg_tot)
    if "UKESM" in file_zg_str:
        file_zg_tot = file_zg_tot.sel(time = np.isin(file_zg_tot['time.dayofyear'], np.arange(147,245)))
    else:
        JJA_extd_daymonth_xr = xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/day_month_1979-2019_JJAextd.nc")['JJA_extd']
        day_month = xr.open_dataset("/rds/general/project/nowack_graven/live/carl/era5/day_month_1979-2019_nolp.nc")['day_month']
        file_zg_tot = file_zg_tot.sel(latitude = np.isin(file_zg_tot['latitude'], lats), 
                                  longitude = np.isin(file_zg_tot['longitude'], lons))
    num_days, num_lats, num_lons = file_zg_tot.shape


    init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling = "pca", "gaussian", 0.5, 50, 1, 0, "linear", 0.1, 0.01, "exponential"
    region, season = "Europe", "JJA_extd"
    domain="EUR"
    samp = int((yrst_train_arr[1]-yrst_train_arr[0])/2)
    colormap_str = "seismic"
    suptitle=f"zg LTDM anom {mdl} {yrst}-{yrend} JJA extd ({units})"
    #all cover different variations on the mltiple hypothesis test applied in Horton et al 2015
    mult_test_method = "fdr_bh"#"k-FWER"#"none"#"fdr_bh"

    if som_yr_train_swap != None:
            dir_str = "/rds/general/project/nowack_graven/live/carl_som_index/data"
            if "UKESM" in file_zg_str:
                mdl_str = "UKESM1-0-LL_piControl"
                swap_str = "_swapERA5"
            elif "era5" in file_zg_str:
                mdl_str = "era5"
                swap_str = "_swapUKESM1-0-LL"
            else:
                print("not UKESM or era5 - cannot define save directories")
                return
            savefig_SOM = (f"{dir_str}/{mdl_str}/{zg_str}/plots/"
                        f"SOM_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.png")
            savefig_trends = (f"{dir_str}/{mdl_str}/{zg_str}/plots/"
                        f"SOMtrends_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.png")
            SOM_trend_str = (f"{dir_str}/{mdl_str}/{zg_str}/"
                        f"SOM_tot_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.nc")
            SOM_data_occ_str = (f"{dir_str}/{mdl_str}/{zg_str}/"
                        f"SOM_data_occur_tot_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst)}-{int(yrend)}_{season}_{zg_str}{swap_str}.nc")


            JJA_days = int(len(np.unique(file_zg_tot['time.dayofyear'])))
            if "era5" in file_zg_str and "msl" not in file_zg_str:
                JJA_days = JJA_days-1 #where there are leap years, need to remove from JJA_days since dayofyear will reflec the different leap year 
            #print(data_yr_reshaped_train)
            print(f"data_yr_reshaped = {data_yr_reshaped}")
            #print("use trained SOM to calculate cv dataset")
            #using the trained SOM, calculate the 
            da_xr_trend, all_occur, som_yr = save_SOM_data(file_zg_tot, yrst, yrend, 
                        season, region, init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling,
                        savefig_SOM, savefig_trends, n_rows, n_columns, samp, zg_str, colormap_str, suptitle, mult_test_method, 
                            data_yr_reshaped, lat_str, lon_str, units, mdl_ens_str, train_input = True, som_yr = som_yr_train_swap, JJA_days=JJA_days, showplot=showplot)
            

            print(f"saving tested data in {SOM_trend_str} \n {SOM_data_occ_str}")
            da_xr_trend.to_netcdf(SOM_trend_str)
            da = xarray.DataArray(all_occur, name = "SOM_data", dims = ("row", "col", "time"))
            da['time'] = file_zg_tot['time']
            da.to_netcdf(SOM_data_occ_str)
        
    else:
        #cross-validation training period
        for yrst_train in yrst_train_arr[:]:
            #print(f"train period {yrst_train}-{yrst_train+yrnum_train}")
            if int(yrst_train) > 0:
                #subselect training and cv periods    
                file_zg_tot_train = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], np.arange(yrst_train, yrst_train+yrnum_train+1), invert=True))
                data_yr_reshaped_train = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time'], file_zg_tot_train['time']))

                file_zg_tot_cv = file_zg_tot.sel(time = np.isin(file_zg_tot['time.year'], np.arange(yrst_train, yrst_train+yrnum_train+1), invert=False))
                data_yr_reshaped_cv = data_yr_reshaped.sel(time = np.isin(data_yr_reshaped['time'], file_zg_tot_cv['time']))        
                if "UKESM" in file_zg_str:
                    #/rds/general/user/cmt3718/home/data/cmip6/SOM/trends/UK-ESM1-0-LL_piControl/zg/crossval/10-fold
                    #/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/zg/crossval/5-fold/
                    savefig_SOM = f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/plots/SOM_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.png"
                    savefig_trends = f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/plots/SOMtrends_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.png"
                    SOM_trend_str_train = f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/SOM_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.nc"
                    SOM_data_occ_str_train = f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/SOM_data_occur_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.nc"
                    SOM_trend_str_cv = f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/SOM_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.nc"
                    SOM_data_occ_str_cv = f"/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/SOM_data_occur_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.nc"

                elif "era5" in file_zg_str:
                    savefig_SOM = f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/plots/SOM_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.png"
                    savefig_trends = f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/plots/SOMtrends_fig_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.png"
                    SOM_trend_str_train = f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/SOM_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.nc"
                    SOM_data_occ_str_train = f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/SOM_data_occur_train_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_not{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.nc"
                    SOM_trend_str_cv = f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/SOM_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.nc"
                    SOM_data_occ_str_cv = f"/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/SOM_data_occur_cv_{mdl_ens_str}_{n_rows}x{n_columns}_{domain}_{int(yrst_train)}-{int(yrst_train+yrnum_train)}_{season}_{zg_str}_LTDManom.nc"
                else:
                    print("not UKESM or era5 - cannot define save directories")
                    return

                JJA_days = int(len(np.unique(file_zg_tot['time.dayofyear'])))
                if "era5" in file_zg_str and "msl" not in file_zg_str:
                    JJA_days = JJA_days-1 #where there are leap years, need to remove from JJA_days since dayofyear will reflec the different leap year 
                #print(data_yr_reshaped_train)
                #load the training dataset
                da_xr_trend_train, all_occur_train, som_yr_train = save_SOM_data(file_zg_tot_train, yrst, yrend, 
                            season, region, init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling,
                            savefig_SOM, savefig_trends, n_rows, n_columns, samp, zg_str, colormap_str, suptitle, mult_test_method, 
                                data_yr_reshaped_train, lat_str, lon_str, units, mdl_ens_str, train_input = False, JJA_days=JJA_days, save_SOM_str=savefig_SOM, showplot=showplot)

                #print("use trained SOM to calculate cv dataset")
                #using the trained SOM, calculate the 
                da_xr_trend_cv, all_occur_cv, som_yr_train = save_SOM_data(file_zg_tot_cv, yrst, yrend, 
                            season, region, init, neigh, std, ep, rad_0, rad_N, rad_cooling, scale_0, scale_N, scale_cooling,
                            savefig_SOM, savefig_trends, n_rows, n_columns, samp, zg_str, colormap_str, suptitle, mult_test_method, 
                                data_yr_reshaped_cv, lat_str, lon_str, units, mdl_ens_str, train_input = True, som_yr = som_yr_train, JJA_days=JJA_days, showplot=showplot)

                print(f"saving training in {SOM_trend_str_train}")
                da_xr_trend_train.to_netcdf(SOM_trend_str_train)
                da_train=xarray.DataArray(all_occur_train, name = "SOM_data", dims = ("row", "col", "time"))
                da_train['time']=file_zg_tot_train['time']
                da_train.to_netcdf(SOM_data_occ_str_train)
                print(f"saving cv in {SOM_trend_str_cv}")
                da_xr_trend_cv.to_netcdf(SOM_trend_str_cv)
                da_cv=xarray.DataArray(all_occur_cv, name = "SOM_data", dims = ("row", "col", "time"))
                da_cv['time']=file_zg_tot_cv['time']
                da_cv.to_netcdf(SOM_data_occ_str_cv)
    return


In [6]:
n_rows, n_columns = 3, 3
file_zg_str_train = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_r180x91.nc"
                     #"/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_sort.nc"
file_zg_str_test = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/"
"500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd.nc")
    
#train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns)
    

In [7]:
arr = sorted(glob.glob("/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/zg/*SOM_data_occur*"))

nrows_todo = [int(file_str.split("_")[10].split("x")[0]) for file_str in arr]
ncols_todo = [int(file_str.split("_")[10].split("x")[1]) for file_str in arr]



In [11]:
rowcols_arr = [(2, 2), (5, 1), (3, 2), (7, 1), (4, 2), (3, 3), (5, 2), (11, 1), (4, 3), (13, 1), (7, 2), (5, 3), (4, 4), (17, 1), (6, 3),
 (19, 1), (5, 4), (7, 3), (11, 2), (23, 1), (6, 4), (5, 5), (13, 2), (9, 3), (7, 4), (29, 1), (6, 5), (31, 1), (8, 4), (33, 1), (17, 2),
               (7, 5), (6, 6), (37, 1), (19, 2), (39, 1), (8, 5), (41, 1)]


for n_rows, n_cols in rowcols_arr[4:]:
        print(f"n_rows, n_columns = {n_rows}, {n_cols}")
        try:
            train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_cols)
        except PermissionError:
            continue

n_rows, n_columns = 4, 2
training
save_SOM_str = /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/plots/SOM_fig_era5_reanal_4x2_EUR_not1979-2019_JJA_extd_z_tot.png
saved trends in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_train_era5_reanal_4x2_EUR_not1979-2019_JJA_extd_z_tot.nc
n_rows, n_columns = 3, 3
training
save_SOM_str = /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/plots/SOM_fig_era5_reanal_3x3_EUR_not1979-2019_JJA_extd_z_tot.png
saved trends in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_train_era5_reanal_3x3_EUR_not1979-2019_JJA_extd_z_tot.nc
saved data_occ in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_data_occur_train_era5_reanal_3x3_EUR_not1979-2019_JJA_extd_z_tot.nc
test the trained SOM on /rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd

/home/cmt3718/anaconda3/envs/odin/lib/python3.6/site-packages/somoclu/train.py:497: RuntimeWarning: invalid value encountered in true_divide
  coord = coord / [self._n_rows - 1, self._n_columns - 1]


save_SOM_str = /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/plots/SOM_fig_era5_reanal_11x1_EUR_not1979-2019_JJA_extd_z_tot.png
saved trends in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_train_era5_reanal_11x1_EUR_not1979-2019_JJA_extd_z_tot.nc
saved data_occ in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_data_occur_train_era5_reanal_11x1_EUR_not1979-2019_JJA_extd_z_tot.nc
test the trained SOM on /rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd.nc
data_yr_reshaped = <xarray.DataArray 'data_yr_reshaped' (time: 9898, latlon: 624)>
[6176352 values with dtype=float32]
Coordinates:
  * time     (time) object 1960-05-27 12:00:00 ... 2060-09-04 12:00:00
  * latlon   (latlon) int64 0 1 2 3 4 5 6 7 ... 616 617 618 619 620 621 622 623
Attributes:
    var:      zg
    anom:     daily mean
saving tes

/home/cmt3718/anaconda3/envs/odin/lib/python3.6/site-packages/somoclu/train.py:497: RuntimeWarning: invalid value encountered in true_divide
  coord = coord / [self._n_rows - 1, self._n_columns - 1]


save_SOM_str = /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/plots/SOM_fig_era5_reanal_13x1_EUR_not1979-2019_JJA_extd_z_tot.png


...

/home/cmt3718/anaconda3/envs/odin/lib/python3.6/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


save_SOM_str = /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/plots/SOM_fig_era5_reanal_8x5_EUR_not1979-2019_JJA_extd_z_tot.png
saved trends in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_train_era5_reanal_8x5_EUR_not1979-2019_JJA_extd_z_tot.nc
saved data_occ in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_data_occur_train_era5_reanal_8x5_EUR_not1979-2019_JJA_extd_z_tot.nc
test the trained SOM on /rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd.nc
data_yr_reshaped = <xarray.DataArray 'data_yr_reshaped' (time: 9898, latlon: 624)>
[6176352 values with dtype=float32]
Coordinates:
  * time     (time) object 1960-05-27 12:00:00 ... 2060-09-04 12:00:00
  * latlon   (latlon) int64 0 1 2 3 4 5 6 7 ... 616 617 618 619 620 621 622 623
Attributes:
    var:      zg
    anom:     daily mean
saving tested

/home/cmt3718/anaconda3/envs/odin/lib/python3.6/site-packages/somoclu/train.py:497: RuntimeWarning: invalid value encountered in true_divide
  coord = coord / [self._n_rows - 1, self._n_columns - 1]


save_SOM_str = /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/plots/SOM_fig_era5_reanal_41x1_EUR_not1979-2019_JJA_extd_z_tot.png
saved trends in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_train_era5_reanal_41x1_EUR_not1979-2019_JJA_extd_z_tot.nc
saved data_occ in /rds/general/project/nowack_graven/live/carl_som_index/data/era5/z/SOM_data_occur_train_era5_reanal_41x1_EUR_not1979-2019_JJA_extd_z_tot.nc
test the trained SOM on /rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd.nc
data_yr_reshaped = <xarray.DataArray 'data_yr_reshaped' (time: 9898, latlon: 624)>
[6176352 values with dtype=float32]
Coordinates:
  * time     (time) object 1960-05-27 12:00:00 ... 2060-09-04 12:00:00
  * latlon   (latlon) int64 0 1 2 3 4 5 6 7 ... 616 617 618 619 620 621 622 623
Attributes:
    var:      zg
    anom:     daily mean
saving tes

/home/cmt3718/anaconda3/envs/odin/lib/python3.6/site-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [15]:
#n_row_vals = [2,2]
#n_col_vals = [1,2]

n_row_vals = [3,5,6,3,7,8,4,9,10,5]
n_col_vals = [1,1,1,2,1,1,2,1,1 ,2]

n_row_vals = range(11,51)
n_columns = 1
for n_rows in n_row_vals:
    print(f"n_rows, n_columns = {n_rows}, {n_columns}")
    train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns)


n_row_vals = range(2,26)
col = 2
for n_rows in n_row_vals:
    print(f"n_rows, n_columns = {n_rows}, {n_columns}")
    train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns)

n_row_vals = range(3,17)
col = 3
for n_rows in n_row_vals:
    print(f"n_rows, n_columns = {n_rows}, {n_columns}")
    train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns)

n_row_vals = range(4,13)
col = 4
for n_rows in n_row_vals:
    print(f"n_rows, n_columns = {n_rows}, {n_columns}")
    train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns)

n_row_vals = range(5,11)
col = 5
for n_rows in n_row_vals:
    print(f"n_rows, n_columns = {n_rows}, {n_columns}")
    train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns)

n_row_vals = range(6,9)
col = 6
for n_rows in n_row_vals:
    print(f"n_rows, n_columns = {n_rows}, {n_columns}")
    train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns)

n_row_vals = range(7,8)
col = 7
for n_rows in n_row_vals:
    print(f"n_rows, n_columns = {n_rows}, {n_columns}")
    train_test_maps(file_zg_str_train, file_zg_str_test, n_rows, n_columns)
    
    

n_rows, n_columns = 11, 1


NameError: name 'file_zg_str_train' is not defined

In [13]:
file_zg_str = ("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_r180x91_reshaped.nc")
file_zg_str = ("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_r180x91.nc")


In [25]:
da=xr.open_dataset("/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/zg/SOM_tot_UKESM1-0-LL_r1i1p1f2_3x3_EUR_1960-2060_JJA_extd_zg_swapERA5.nc")
#['__xarray_dataarray_variable__']

In [330]:

#("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/psl/"
# "psl_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd_reshaped.nc")
file_zg_str = ("/rds/general/user/cmt3718/home/data/cmip6/UKESM1-0-LL/piControl/r1i1p1f2/zg/"
"500zg_day_UKESM1-0-LL_piControl_r1i1p1f2_gn_19600101-20601230_r180x91_LTDManom_EUR2_JJAextd_dtrnd.nc")
mip = "cmip6"



file_zg_str = "/rds/general/project/nowack_graven/live/carl/era5/mean_sea_level_pressure/mslp_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom.nc"
file_zg_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_sort.nc"


mip = "era5"
ncol = 1
nrow_vals, ncol_vals = [3], [3]
for nrow, ncol in zip(nrow_vals, ncol_vals):
    print(f"nrow, ncol = {nrow, ncol}")
    for yrnum_train in [4]:
        print(f"yrnum_train = {yrnum_train}")
        SOM_calc_samp_swap(file_zg_str, nrow, ncol, yrnum_train)

nrow, ncol = (3, 3)
yrnum_train = 4


AttributeError: 'Dataset' object has no attribute 'shape'

In [98]:
z_lonadj_eur = z_lonadj.sel(lon=np.arange(-10,42,2), lat=np.arange(30,78,2))

In [248]:
def da_lon_adj(da):
    """
    Reset the longitude coordinates if they are defined from 0-360 E to -180 - 180 E.
    """
    #identify label for the longitude coordinate
    if "longitude" in str(da.coords):
        lon_str = "longitude"
    else:
        lon_str = "lon"
    if int(da[lon_str].max()>180):
        #calculate longitude resolution
        lon_diff = abs(int(da[lon_str][1]-da[lon_str][0]))
        #redefine 0-360 degrees E to -180 - +180 deg E  #+lon_diff/2
        print(f"da[lon_str] = {da[lon_str]}")
        
        lon_adj = da[lon_str]*(da[lon_str]<(180+lon_diff/2))+((da[lon_str]-360)*(da[lon_str]>180))
        print(f"lon_adj = {lon_adj}")
        da[lon_str] = lon_adj
        # number of indices that lat idx should be mvoed by
        if da[lon_str][0] == 0:
            lat_idx_roll = int(len(da[lon_str])/2)
        else:
            print("ERROR - coordiante system doesn't start from 0 degrees east")
            return 1
    
        if lon_str == "lon":
            da_roll = da.roll(lon = lat_idx_roll, roll_coords=True)
        if lon_str == "longitude":
            da_roll = da.roll(longitude = lat_idx_roll, roll_coords=True)     
        #check that the output longitude coordinates are in ascending order
        if False not in np.array(da[lon_str].values == da[lon_str].sortby(lon_str, ascending=True).values):
            return da_roll
        else:
            print("ERROR - longitude coordinates are not in ascending order")
            return 1
    else:
        return da

In [231]:
file_zg_str = "/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_sort.nc"

z_1x1=xr.open_dataset(file_zg_str)['z']


z_1x1_lonadj=SOM_fn.da_lon_adj(z_1x1)


In [250]:
z_2x2=xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_1x1_1979-2019_daymean_LTDManom_nolp_r180x91.nc")['z']
#z_1x1=xr.open_dataset("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_1x1_1979-2019_daymean_LTDManom_nolp.nc")['z']



z_2x2_lonadj=SOM_fn.da_lon_adj(z_2x2)
#z_1x1_lonadj=SOM_fn.da_lon_adj(z_1x1)

#cdo remapbil,r180x91 500zg_1x1_1979-2019_daymean_LTDManom_nolp.nc 500zg_1x1_1979-2019_daymean_LTDManom_nolp_r180x91.nc


In [225]:
z_2x2_lonadj=da_lon_adj(z_2x2)
#z_1x1_lonadj=da_lon_adj(z_1x1[0])

In [251]:
z_2x2_lonadj_eur = z_2x2_lonadj.sel(lat = np.arange(30,78,2), lon = np.arange(-10, 42, 2))

In [252]:

def sel_JJAextd(da):
    """
    Select the extened JJA period
    """
    da_MJJAS = da.sel(time = np.isin(da['time.month'], range(5,10)))
    da_JJAextdS = da_MJJAS.sel(time=~(((da_MJJAS.time.dt.month == 5) & (da_MJJAS.time.dt.day < 28))))
    da_JJAextd = da_JJAextdS.sel(time=~(((da_JJAextdS.time.dt.month == 9) & (da_JJAextdS.time.dt.day > 4))))
    return da_JJAextd
    
z_2x2_lonadj_eur_JJAextd = sel_JJAextd(z_2x2_lonadj_eur)

In [256]:
def fldmean_xr(xr_da):
    """
    Calculate the area averaged mean from an xarray DataArray with coordinates time, latitude, longitude
    will need to be adapted if data is not 3d e.g. if there are other levels
    """
    if "latitude" in str(xr_da):
        lat_str, lon_str = "latitude", "longitude"
    else:
        lat_str, lon_str = "lat", "lon"
    import numpy as np
    import xarray as xr
    #print(lat_str)
    
    #multiply the ataarray by the relevant weights using cos(latitude)
    xr_da_latcorr = (xr_da*np.cos(np.radians(xr_da[lat_str])))    
    #calculate the weighted mean by summing across all the weighted values and dividing by the total sum of all weights appliced
    if xr.ufuncs.isnan(xr_da[0]).sum() > 0:
        print("Masked")
        #calculating the weighted mean for a masked array
        isnan = xr.ufuncs.isnan(xr_da[0])
        xr_da_latcorr = (xr_da*np.cos(np.radians(xr_da[lat_str])))
        weights_sum = ((isnan==0)*np.cos(np.radians(xr_da[lat_str]))).sum()
        xr_da_fldmean_latcorr = xr_da_latcorr.sum(axis=1, skipna=True).sum(axis=1, skipna=True)/weights_sum
    
    
    if len(xr_da.shape) == 3:
        xr_da_fldmean_latcorr = xr_da_latcorr.sum(axis=1).sum(axis=1)/(np.cos(np.radians(xr_da_latcorr[lat_str])).sum()*xr_da_latcorr[lon_str].shape[0])
    if len(xr_da.shape) == 2:
        xr_da_fldmean_latcorr = xr_da_latcorr.sum(axis=1).sum(axis=0)/(np.cos(np.radians(xr_da_latcorr[lat_str])).sum()*xr_da_latcorr[lon_str].shape[0])        
    return xr_da_fldmean_latcorr

In [257]:
z_2x2_lonadj_eur_JJAextd_fldmean = fldmean_xr(z_2x2_lonadj_eur_JJAextd)


#.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/500zg_1x1_1979-2019_JJAextd_daymean_LTDManom_r180x91_EUR.nc")

In [267]:
slope, intercept, rval, pval, stderr = scipy.stats.linregress(range(len(z_2x2_lonadj_eur_JJAextd_fldmean)), z_2x2_lonadj_eur_JJAextd_fldmean)

dtrnd_arr = []
for i in range(len(z_2x2_lonadj_eur_JJAextd_fldmean)):
    z_2x2_lonadj_eur_JJAextd_dtrnd = z_2x2_lonadj_eur_JJAextd[i] - intercept - slope*i
    dtrnd_arr.append(z_2x2_lonadj_eur_JJAextd_dtrnd)
dtrnd_arr_concat = xr.concat(dtrnd_arr, dim = "time")    


In [278]:
zg_dtrnd_arr_concat = xr.DataArray(dtrnd_arr_concat, attrs = {"dtrnd": "time", "period": "JJA_extd", "model": "era5", "variable": "z500", "domain": "Europe"})

In [282]:
zg_dtrnd_arr_concat.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_r180x91.nc")
                              #500zg_1x1_1979-2019_JJAextd_daymean_LTDManom_r180x91_EUR_.nc")

In [284]:
zg_dtrnd_arr_concat_reshaped = zg_dtrnd_arr_concat.values.reshape(4100,24*26) 

In [293]:
zg_dtrnd_arr_concat_reshaped_da = xr.DataArray(zg_dtrnd_arr_concat_reshaped, dims = ({"time": zg_dtrnd_arr_concat['time'], "lat_lon": np.arange(624)}),
                                               attrs = {"dtrnd": "time", "period": "JJA_extd", "model": "era5", "variable": "z500", "domain": "Europe"})

In [294]:
zg_dtrnd_arr_concat_reshaped_da['time'] = zg_dtrnd_arr_concat['time']

In [295]:
zg_dtrnd_arr_concat_reshaped_da.to_netcdf("/rds/general/project/carl_phd/live/carl/data/era5/day/zg/LTDM/z_timedtrnd_ERA5_1979-2019_EUR_JJAextd_LTDMdaymean_anom_r180x91_reshaped.nc")